#Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
low_memory=False

In [ ]:
train_path = '/content/drive/MyDrive/Sales Forecast/train.csv'
test_path = '/content/drive/MyDrive/Sales Forecast/test.csv'
store_path = '/content/drive/MyDrive/Sales Forecast/store.csv'

In [ ]:
train = pd.read_csv(train_path, parse_dates=['Date'])
test = pd.read_csv(test_path, parse_dates=['Date'])
store = pd.read_csv(store_path)

/tmp/ipython-input-5-3583835014.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path, parse_dates=['Date'])


In [ ]:
train.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0


In [ ]:
test.isnull().sum()

,0
Id,0
Store,0
DayOfWeek,0
Date,0
Open,11
Promo,0
StateHoliday,0
SchoolHoliday,0


In [ ]:
store.isnull().sum()

,0
Store,0
StoreType,0
Assortment,0
CompetitionDistance,3
CompetitionOpenSinceMonth,354
CompetitionOpenSinceYear,354
Promo2,0
Promo2SinceWeek,544
Promo2SinceYear,544
PromoInterval,544


##Missing Values

Checking to see when a store does not participate in a promo

In [ ]:
mask = store['Promo2'] == 0
store.loc[mask, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].isnull().sum()

,0
Promo2SinceWeek,544
Promo2SinceYear,544
PromoInterval,544


We will fill these specific NA's with 0, as it makes sense that when a store does not participate in a promo the values of PromoSince & Intervals are 0

In [ ]:
store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(0).astype(int)
store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(0).astype(int)

Label Encoding PromoInterval: I replaced month-based string patterns with numerical categories using the following mapping:
   - `'Jan,Apr,Jul,Oct'` → `1`
   - `'Feb,May,Aug,Nov'` → `2`
   - `'Mar,Jun,Sept,Dec'` → `3`
   - No Interval  → `0`


In [ ]:
promo_map = {
    'Jan,Apr,Jul,Oct': 1,
    'Feb,May,Aug,Nov': 2,
    'Mar,Jun,Sept,Dec': 3
}
store['PromoInterval'] = store['PromoInterval'].map(promo_map)
store['PromoInterval']= store['PromoInterval'].astype('Int64')
store['PromoInterval'].fillna(0, inplace=True)


/tmp/ipython-input-11-85895242.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  store['PromoInterval'].fillna(0, inplace=True)


Any store with 0 competition distance means that it does not have competition, hence we impute the competitionopensince with 0

In [ ]:
mask = store['CompetitionDistance'].isna()
store.loc[mask, ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']] = 0

Filled Missing Values with -1:  We imputed `NaN` values in the following columns using `-1` as a sentinel value to preserve the fact that the information was missing:
   - `CompetitionOpenSinceMonth`
   - `CompetitionOpenSinceYear`

In [ ]:
#Impute missing values with sentinel (-1) to preserve interpretability
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(-1).astype(int)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(-1).astype(int)

In [ ]:
store.isnull().sum()

,0
Store,0
StoreType,0
Assortment,0
CompetitionDistance,0
CompetitionOpenSinceMonth,0
CompetitionOpenSinceYear,0
Promo2,0
Promo2SinceWeek,0
Promo2SinceYear,0
PromoInterval,0


Since there are no missing values in all 3 datasets, I will move on to label encoding and any other preprocessing steps neeeded.

##Label Encoding

In [ ]:
print("\nData types in train DataFrame:")
print(train.dtypes)
print("\nData types in test DataFrame:")
print(test.dtypes)
print("\nData types in store DataFrame:")
store.dtypes


Data types in train DataFrame:
Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

Data types in test DataFrame:
Id                        int64
Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Open                    float64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

Data types in store DataFrame:


,0
Store,int64
StoreType,object
Assortment,object
CompetitionDistance,float64
CompetitionOpenSinceMonth,int64
CompetitionOpenSinceYear,int64
Promo2,int64
Promo2SinceWeek,int64
Promo2SinceYear,int64
PromoInterval,Int64


The `LabelEncoder` was used to convert the categorical values in the 'StoreType' and 'Assortment' columns into numerical labels. The mapping is as follows:

**StoreType:**
- 'a' is mapped to 0
- 'b' is mapped to 1
- 'c' is mapped to 2
- 'd' is mapped to 3

**Assortment:**
- 'a' is mapped to 0
- 'b' is mapped to 1
- 'c' is mapped to 2

In [ ]:
le = LabelEncoder()

store['StoreType'] = le.fit_transform(store['StoreType'])
store['Assortment'] = le.fit_transform(store['Assortment'])

print(store[['StoreType', 'Assortment']].head())


   StoreType  Assortment
0          2           0
1          0           0
2          0           0
3          2           2
4          0           0


In [ ]:
sales_train = train.merge(store, on='Store', how='left')
sales_test = test.merge(store, on='Store', how='left')

In [ ]:
print("\nData types in train DataFrame:")
print(sales_train.dtypes)
print("\nData types in test DataFrame:")
print(sales_test.dtypes)


Data types in train DataFrame:
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                             int64
Assortment                            int64
CompetitionDistance                 float64
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                int64
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                         Int64
dtype: object

Data types in test DataFrame:
Id                                    int64
Store                                 int64
DayOfWeek                             int64

The `StateHoliday` column was label encoded using the following mapping:

**StateHoliday:**
- '0' (no holiday) is mapped to 0
- 'a' (public holiday) is mapped to 1
- 'b' (Easter holiday) is mapped to 2
- 'c' (Christmas) is mapped to 3

In [ ]:
holiday_map = {
    '0': 0,  # '0' as string
    'a': 1,
    'b': 2,
    'c': 3
}

sales_train['StateHoliday'] = sales_train['StateHoliday'].replace(0, '0')
sales_test['StateHoliday'] = sales_test['StateHoliday'].replace(0, '0')


sales_train['StateHoliday'] = sales_train['StateHoliday'].map(holiday_map).astype(int)
sales_test['StateHoliday'] = sales_test['StateHoliday'].map(holiday_map).astype(int)

In [ ]:
print("\nData types in train DataFrame:")
print(sales_train.dtypes)
print("\nData types in test DataFrame:")
print(sales_test.dtypes)


Data types in train DataFrame:
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                          int64
SchoolHoliday                         int64
StoreType                             int64
Assortment                            int64
CompetitionDistance                 float64
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                int64
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                         Int64
dtype: object

Data types in test DataFrame:
Id                                    int64
Store                                 int64
DayOfWeek                             int64

In [ ]:
print("\nUnique values in 'Open' column of sales_test:")
print(sales_test['Open'].unique())


Unique values in 'Open' column of sales_test:
[ 1. nan  0.]


In [ ]:
print("\nValue counts for 'Open' column in sales_test:")
print(sales_test['Open'].value_counts())


Value counts for 'Open' column in sales_test:
Open
1.0    35093
0.0     5984
Name: count, dtype: int64


If Open is missing in the test set, assume the store is open (Open = 1), unless it’s a Sunday (DayOfWeek == 7), in which case default to closed (Open = 0).

In [ ]:
sales_test['Open'] = test.apply(
    lambda row: 0 if pd.isna(row['Open']) and row['DayOfWeek'] == 7 else (1 if pd.isna(row['Open']) else row['Open']),
    axis=1
)

In [ ]:
sales_test['Open'] = sales_test['Open'].astype(int)

In [ ]:
print("\nData types in train DataFrame:")
print(sales_train.dtypes)
print("\nData types in test DataFrame:")
print(sales_test.dtypes)


Data types in train DataFrame:
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                          int64
SchoolHoliday                         int64
StoreType                             int64
Assortment                            int64
CompetitionDistance                 float64
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                int64
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                         Int64
dtype: object

Data types in test DataFrame:
Id                                    int64
Store                                 int64
DayOfWeek                             int64

In [ ]:
train_cols = set(sales_train.columns)
test_cols = set(sales_test.columns)

# Find columns in training set that are not in test set
cols_in_train_not_in_test = list(train_cols - test_cols)

print("Columns in training set not present in test set:")
cols_in_train_not_in_test

Columns in training set not present in test set:


['Sales', 'Customers']

we will drop customers as it is not in testing environment

In [ ]:
sales_train = sales_train.drop('Customers', axis=1)

tranforming sales

In [ ]:
sales_train['Sales'] = np.log1p(sales_train['Sales'])

In [ ]:
sales_train['DateInt'] = (sales_train['Date'] - pd.to_datetime("2013-01-01")).dt.days

##Saving Training & Testing

In [ ]:
sales_train.to_csv('/content/drive/MyDrive/Sales Forecast/train_processed1.csv', index=False)
sales_test.to_csv('/content/drive/MyDrive/Sales Forecast/test_processed1.csv', index=False)